In [69]:
from gurobipy import *
import pandas as pd

In [71]:
try:

    plants = ['PlantOne','PlantTwo']

    regions = ['RegionOne','RegionTwo','RegionThree']

    cost = {
        ('PlantOne','RegionOne'): 250,
        ('PlantOne','RegionTwo'): 325,
        ('PlantOne','RegionThree'): 445,
        ('PlantTwo','RegionOne'): 275,
        ('PlantTwo','RegionTwo'): 260,
        ('PlantTwo','RegionThree'): 460
    }

    supply = {
        ('PlantOne'): 100,
        ('PlantTwo'): 125
    }

    demand = {
        ('RegionOne'): 80,
        ('RegionTwo'): 40,
        ('RegionThree'): 80
    }

    # Create a new model
    m = Model("transport_problem_1")

    # Create variables
    flow = {}
    for f in plants:
        for s in regions:
            flow[f,s] = m.addVar(obj=cost[f,s], name='flow_%s_%s' % (f, s))

    # Integrate new variables
    m.update()

    # Add supply constraints
    for f in plants:
        m.addConstr(quicksum(flow[f,s] for s in regions) <= supply[f], 'supply_%s' % (f))
    
    # Add demand constraints
    for s in regions:
        m.addConstr(quicksum(flow[f,s] for f in plants) >= demand[s], 'demand_%s' % (s))

    # Optimize the model. The default ModelSense is to is to minimize the objective, which is what we want.
    m.optimize()

    # Print solution
    if m.status == GRB.status.OPTIMAL:
        print ('\nOptimal flows :')
        for f in plants:
            for s in regions:
                print (f, '->', s, ':', flow[f,s].x)

except GurobiError:
    print ('Error reported')



Optimize a model with 5 rows, 6 columns and 12 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+02, 5e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+01, 1e+02]
Presolve time: 0.00s
Presolved: 5 rows, 6 columns, 12 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   2.000000e+02   0.000000e+00      0s
       4    6.6900000e+04   0.000000e+00   0.000000e+00      0s

Solved in 4 iterations and 0.01 seconds
Optimal objective  6.690000000e+04

Optimal flows :
PlantOne -> RegionOne : 80.0
PlantOne -> RegionTwo : 0.0
PlantOne -> RegionThree : 20.0
PlantTwo -> RegionOne : 0.0
PlantTwo -> RegionTwo : 40.0
PlantTwo -> RegionThree : 60.0
